In [ ]:
import sys
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt
from PyQt5.QtWidgets import (
    QApplication, QLabel, QMainWindow, QVBoxLayout, QWidget,
    QLineEdit, QPushButton, QHBoxLayout, QScrollArea
)
from PyQt5.QtGui import QPixmap, QImage, QPainter, QColor, QPen
from PyQt5.QtCore import Qt, QEvent

In [ ]:
# Load the GeoTIFF file
file_path = "/kaggle/input/geotiff/testGeoTiff.tif"
dataset = gdal.Open(file_path)

# Get and display image resolution (width and height)
cols = dataset.RasterXSize
rows = dataset.RasterYSize
print("Image Resolution: {} x {}".format(cols, rows))

# Get and display number of bands
num_bands = dataset.RasterCount
print("Number of Bands:", num_bands)

# Read the first three bands (assuming an RGB composite)
band1 = dataset.GetRasterBand(1).ReadAsArray()
band2 = dataset.GetRasterBand(2).ReadAsArray()
band3 = dataset.GetRasterBand(3).ReadAsArray()

# Stack the bands into an RGB image
rgb_image = np.dstack((band1, band2, band3))

# Display the image inline
plt.figure(figsize=(10, 10))
plt.imshow(rgb_image)
plt.title("Landsat Image")
plt.axis("off")
plt.show()